# 1. Brownian path 1

In [ ]:
using Plots, Statistics

In [ ]:
T=1
N=500
dt= T/N
dW=zeros(N,1)
W=zeros(N,1);

dW[1]= (sqrt(dt) * randn())
W[1]=dW[1]

for j= 2:N
    dW[j]= (sqrt(dt)*randn())
    W[j]=W[j-1] + dW[j]
end

plot(LinRange(0,T,N),W)

In [ ]:
randn()

# 2. Brownian path 2: vectorized

In [ ]:
T=1
N=500
dt= T/N

dW = sqrt(dt)* randn(N,1)
W = cumsum(dW;dims=1)

plot(LinRange(0,T,N),W)

# 3. Function along a Brownian path 

In [ ]:
] add Statistics

In [ ]:
using Statistics, LaTeXStrings

In [ ]:
T=1 ;N=500 ;dt= T/N ;t= dt:dt:1
M=1000
dW= sqrt(dt)*randn(N,M)
W = cumsum(dW,dims=1)
U = exp.(repeat(t,1,M)+0.5 *W)
Umean=mean(U,dims=2)
plot(t,Umean)
plot!(t,U[:,1:5])
xlabel!(L"t")
ylabel!(L"U(t)")
averr=norm()

# 4. Ito and Stratonovich integral

In [ ]:
T=1 ;N=500 ;dt= T/N ;t= dt:dt:1
M=1000
dW= sqrt(dt)*randn(N)
W = cumsum(dW,dims=1)

ito= sum([0 ; W[1:end-1]].* dW)
strat = sum( ( 0.5 * (([0 ; W[1:end-1]]+W)  + 0.5*sqrt(dt)*randn(N)) .*dW ))

itoerr = abs(ito - 0.5*(W[end]^2-T)); straterr = abs(strat - 0.5*(W[end]^2))
(itoerr,straterr)

# 5. Euler-Maruyama method on linear SDE

In [ ]:
λ = 2
μ = 1
X0=1
T = 1; N = 2^8; dt = 1/N
dW = sqrt(dt)*randn(N)
W = cumsum(dW)

Xtrue = X0*exp.((λ - 0.5*μ^2)*(dt:dt:T)+μ*W)
plot(0:dt:T,[X0;Xtrue])

R=4 ; Dt = R*dt; L = N/R |> Int
Xem = zeros(L )
Xtemp = copy(X0);
for j = 1:L
    Winc = sum(dW[(R*(j-1)+1:R*j) |>Array{Int}]);
    Xtemp = Xtemp + Dt*λ + μ*Xtemp*Winc;
    Xem[j]=Xtemp
end

plot!(0:Dt:T, [X0;Xem])



In [ ]:
emerr = abs(Xem[end]-Xtrue[end])



# 6. Test strong convergence of EM



In [ ]:
λ = 2
μ = 1
Xzero=1
T = 1; N = 2^8; dt = 1/N
M=1000
Xerr= zeros(5,M)
for s = 1:M
    dW = sqrt(dt)*randn(N)
    W=cumsum(dW)
    Xtrue = Xzero * exp((λ-0.5μ^2)+μ*W[end])
    for p = 1:5
        R = 2^(p-1); Dt = R*dt; L = N/R
        Xtemp = Xzero;
        for j = 1:L
            Winc = sum(dW[(R*(j-1)+1:R*j)|>Array{Int64}])
            Xtemp = Xtemp + Dt*λ*Xtemp + μ*Xtemp*Winc
        end
        Xerr[p,s] = abs(Xtemp - Xtrue)
    end
end

In [ ]:
Dtvals = dt*(2 .^(0:4))
p1 = plot(Dtvals,mean(Xerr,dims=2), xaxis=:log, yaxis=:log)
p2 = plot!(Dtvals, (Dtvals.^(.5)) , xaxis=:log, yaxis=:log)


# 7. EMweak

In [ ]:
λ = 2 ; μ = 1 ; Xzero=1
T = 1 ; M=50000
Xem = zeros(5)
for p = 1:5
    Dt = 2. ^(p-10); L = T/Dt
    Xtemp = Xzero*ones(M) #?
    for j = 1:L
        Winc = sqrt(Dt)*randn(M)
        Xtemp = Xtemp + Dt*λ*Xtemp + μ*Xtemp .* Winc
    end
    Xem[p] = mean(Xtemp)
end
Xerr = abs.(Xem .- exp(λ))

Dtvals = 2.0 .^((1:5) .- 10)

p1 = plot(Dtvals,Xerr , xaxis=:log, yaxis=:log)
p2 = plot!(Dtvals, Dtvals ,xaxis=:log, yaxis=:log)



# 8.Strong convergence of Milstein 

In [ ]:
r = 2; K = 1; β = 0.25; X0 = 0.5;
T = 1; N = 2^(11); dt = T/N;
M = 500;
R = [1; 16; 32; 64; 128]
dW = sqrt(dt)*randn(M,N)
Xmil = zeros(M,5)
#record=[]
for p = 1:5
    Dt = R[p]*dt; L = N/R[p]
    Xtemp = X0 * ones(M)
    for  j = 1:L
         Winc = sum(dW[:,(R[p]*(j-1)+1:R[p]*j)|>Array{Int64}],dims=2)
         Xtemp = Xtemp + Dt*r*Xtemp.*(K .- Xtemp) + β * (Xtemp .* Winc) +  (0.5 * β.^2.0 ).*(Xtemp .* (Winc.^2 .- Dt))
         #push!(record,Xtemp)
    end
    Xmil[:,p] = Xtemp
end



In [ ]:
Xref = Xmil[:,1]
Xerr = abs.(Xmil[:,2:5] - repeat(Xref,1,4))
Xmean = mean(Xerr,dims = 1)'
Dtvals = dt*R[2:5]
plot(Dtvals,Xmean, xaxis =:log,yaxis =:log)
plot(Dtvals,Dtvals,)

In [ ]:
Xmean

# Mean-square and asymptotic ability test for E-M

dX=$\lambda$ * X dt + $\mu$ * X dW
X(0) = 1

$\lambda, \mu$ =Const

In [3]:
T = 20.; M = 50000; X0 = 1.
λ = -3.; μ = sqrt(3);

In [ ]:
for k = 1:3
    Dt = 2. .^(1-k)
    N = (T/Dt)|> Int64
    Xms = zeros(N) ; Xtemp = X0*ones(M);
    for j = 1:N
        Winc = sqrt(Dt)*randn(M)
        Xtemp = Xtemp + (Dt*λ)*Xtemp + μ*(Xtemp.*Winc)
        Xms[j] = mean(Xtemp.^2)
    end
    plot((0:Dt:T),[X0;Xms],yaxis=:log)
end